# EXPOLARATORY ANALYSIS
In this jupyter notebook we will expolore the dataset obtained. In particular we'll focus on: 
1. **Seasonal plot**: basic plotting, year-wise box plot for trend, month_wise box plot for seasonality,
2. **A few metrics**:  average daily yearly and quaterly, mean and std, median close price and 10th percentile and 90th percentile
3. **Correlation and autocorrelation on PCT change**
4. **Test for random walk by applying regression model**

## Seasonal plot

In [1]:
print("Ciao")

Ciao
